
# Exploratory Data Analysis (EDA)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
from scipy import stats as scipy_stats

warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("="*80)
print("STEP 4: SIMPLIFIED EXPLORATORY DATA ANALYSIS")
print("Deep Dive into Data Patterns and Relationships")
print("="*80)


In [ ]:
def load_processed_data():
    """Load the processed dataset with engineered features"""
    print("\n📊 STEP 1: LOADING PROCESSED DATA")
    print("-" * 50)
    
    try:
        # Load the cleaned dataset
        df = pd.read_excel('Data/processed/cleaned_dataset_with_engineered_features.xlsx')
        print("✅ Processed dataset loaded successfully!")
        
        # Load variable grouping table
        grouping_df = pd.read_csv('Data/processed/variable_grouping_table.csv')
        print("✅ Variable grouping table loaded successfully!")
        
        print(f"📏 Dataset shape: {df.shape[0]} rows × {df.shape[1]} columns")
        
        return df, grouping_df
        
    except Exception as e:
        print(f"❌ Error loading processed data: {e}")
        return None, None


In [ ]:
def analyze_domain_variables(df, grouping_df, domain_name):
    """Analyze variables in a specific domain"""
    print(f"\n📊 ANALYZING {domain_name.upper()} VARIABLES")
    print("-" * 50)
    
    # Get variables in this domain
    domain_vars = grouping_df[grouping_df['Domain'] == domain_name]['Variable'].tolist()
    domain_vars = [var for var in domain_vars if var in df.columns]
    
    print(f"📊 {domain_name} variables: {len(domain_vars)}")
    if domain_vars:
        print(f"Variables: {domain_vars[:10]}{'...' if len(domain_vars) > 10 else ''}")
    
    if not domain_vars:
        print(f"⚠️  No {domain_name} variables found")
        return {}
    
    # Analyze each variable
    domain_analysis = {}
    
    for var in domain_vars:
        if df[var].dtype in ['int64', 'float64']:
            # Continuous variable analysis
            stats_dict = {
                'mean': df[var].mean(),
                'std': df[var].std(),
                'min': df[var].min(),
                'max': df[var].max(),
                'median': df[var].median(),
                'missing': df[var].isnull().sum(),
                'unique_values': df[var].nunique()
            }
        else:
            # Categorical variable analysis
            value_counts = df[var].value_counts()
            stats_dict = {
                'value_counts': value_counts.to_dict(),
                'missing': df[var].isnull().sum(),
                'unique_values': df[var].nunique(),
                'most_common': value_counts.index[0] if len(value_counts) > 0 else None
            }
        
        domain_analysis[var] = stats_dict
    
    # Display summary
    print(f"\n📈 {domain_name} Summary:")
    for var, stats in list(domain_analysis.items())[:5]:  # Show first 5
        print(f"\n  {var}:")
        if 'mean' in stats:
            print(f"    Mean: {stats['mean']:.2f}, Std: {stats['std']:.2f}")
            print(f"    Range: {stats['min']:.2f} - {stats['max']:.2f}")
        else:
            print(f"    Unique values: {stats['unique_values']}")
            print(f"    Most common: {stats['most_common']}")
    
    if len(domain_analysis) > 5:
        print(f"\n  ... and {len(domain_analysis) - 5} more variables")
    
    return domain_analysis


In [ ]:
def correlation_analysis_with_birthweight(df):
    """Perform correlation analysis with birthweight"""
    print("\n📊 STEP 2: CORRELATION ANALYSIS WITH BIRTHWEIGHT")
    print("-" * 50)
    
    if 'f1_bw' not in df.columns:
        print("❌ f1_bw variable not found")
        return {}
    
    # Get numerical variables
    numerical_vars = df.select_dtypes(include=[np.number]).columns.tolist()
    
    # Remove constant columns and birthweight itself
    numerical_vars = [var for var in numerical_vars if df[var].nunique() > 1 and var != 'f1_bw']
    
    print(f"📊 Analyzing correlations with {len(numerical_vars)} numerical variables")
    
    # Calculate correlations with birthweight
    correlations = {}
    for var in numerical_vars[:100]:  # Limit to first 100 variables for performance
        if var in df.columns:
            try:
                # Get common non-null values
                common_idx = df[var].dropna().index.intersection(df['f1_bw'].dropna().index)
                if len(common_idx) > 10:  # Need at least 10 observations
                    corr_coef, p_value = scipy_stats.pearsonr(df.loc[common_idx, var], df.loc[common_idx, 'f1_bw'])
                    correlations[var] = {
                        'correlation': corr_coef,
                        'p_value': p_value,
                        'abs_correlation': abs(corr_coef)
                    }
            except:
                continue
    
    # Sort by absolute correlation
    sorted_correlations = sorted(correlations.items(), key=lambda x: x[1]['abs_correlation'], reverse=True)
    
    # Display top correlations
    print("\n📈 Top 20 Correlations with Birthweight:")
    print("Variable".ljust(30) + "Correlation".ljust(15) + "P-value".ljust(15) + "Significance")
    print("-" * 80)
    
    for var, stats in sorted_correlations[:20]:
        significance = "***" if stats['p_value'] < 0.001 else "**" if stats['p_value'] < 0.01 else "*" if stats['p_value'] < 0.05 else ""
        print(f"{var[:29].ljust(30)}{stats['correlation']:8.4f}     {stats['p_value']:8.4f}     {significance}")
    
    return correlations


In [ ]:
def risk_factor_analysis(df):
    """Identify key risk factors for low birthweight"""
    print("\n📊 STEP 3: RISK FACTOR ANALYSIS FOR LOW BIRTHWEIGHT")
    print("-" * 50)
    
    if 'LBW_flag' not in df.columns:
        print("❌ LBW_flag not found in dataset")
        return {}
    
    # Get numerical variables for analysis
    numerical_vars = df.select_dtypes(include=[np.number]).columns.tolist()
    numerical_vars = [var for var in numerical_vars if var not in ['LBW_flag', 'f1_bw'] and df[var].nunique() > 1]
    
    print(f"📊 Analyzing {len(numerical_vars)} variables for LBW risk factors")
    
    risk_factors = {}
    
    for var in numerical_vars[:50]:  # Limit to first 50 variables for performance
        if var in df.columns:
            try:
                # Compare means between LBW and normal birthweight groups
                lbw_group = df[df['LBW_flag'] == 1][var].dropna()
                normal_group = df[df['LBW_flag'] == 0][var].dropna()
                
                if len(lbw_group) > 5 and len(normal_group) > 5:
                    # T-test
                    t_stat, p_value = scipy_stats.ttest_ind(lbw_group, normal_group)
                    
                    # Effect size (Cohen's d)
                    pooled_std = np.sqrt(((len(lbw_group) - 1) * lbw_group.std()**2 + 
                                        (len(normal_group) - 1) * normal_group.std()**2) / 
                                       (len(lbw_group) + len(normal_group) - 2))
                    cohens_d = (lbw_group.mean() - normal_group.mean()) / pooled_std
                    
                    risk_factors[var] = {
                        'lbw_mean': lbw_group.mean(),
                        'normal_mean': normal_group.mean(),
                        'difference': lbw_group.mean() - normal_group.mean(),
                        't_statistic': t_stat,
                        'p_value': p_value,
                        'cohens_d': cohens_d,
                        'effect_size': 'large' if abs(cohens_d) > 0.8 else 'medium' if abs(cohens_d) > 0.5 else 'small'
                    }
            except:
                continue
    
    # Sort by p-value
    sorted_risk_factors = sorted(risk_factors.items(), key=lambda x: x[1]['p_value'])
    
    # Display significant risk factors
    print("\n📈 Significant Risk Factors for Low Birthweight (p < 0.05):")
    print("Variable".ljust(30) + "LBW Mean".ljust(12) + "Normal Mean".ljust(12) + "Difference".ljust(12) + "P-value".ljust(12) + "Effect Size")
    print("-" * 90)
    
    significant_factors = 0
    for var, stats in sorted_risk_factors:
        if stats['p_value'] < 0.05:
            significant_factors += 1
            print(f"{var[:29].ljust(30)}{stats['lbw_mean']:8.2f}    {stats['normal_mean']:8.2f}    {stats['difference']:8.2f}    {stats['p_value']:8.4f}    {stats['effect_size']}")
    
    print(f"\n📊 Found {significant_factors} significant risk factors")
    
    return risk_factors


In [ ]:
def create_eda_plots(df, grouping_df):
    """Create EDA visualization plots"""
    print("\n📊 STEP 4: CREATING EDA PLOTS")
    print("-" * 50)
    
    # Create plots directory
    plots_dir = Path('PLOTS')
    plots_dir.mkdir(exist_ok=True)
    
    # 1. Birthweight distribution by sex
    print("📊 Creating birthweight distribution plots...")
    
    if 'f1_sex' in df.columns and 'f1_bw' in df.columns:
        plt.figure(figsize=(15, 10))
        
        # Subplot 1: Overall distribution
        plt.subplot(2, 3, 1)
        plt.hist(df['f1_bw'], bins=30, alpha=0.7, edgecolor='black', color='skyblue')
        plt.axvline(2500, color='red', linestyle='--', label='LBW threshold (2500g)')
        plt.xlabel('Birthweight (grams)')
        plt.ylabel('Frequency')
        plt.title('Overall Birthweight Distribution')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Subplot 2: By sex
        plt.subplot(2, 3, 2)
        sex_1_data = df[df['f1_sex'] == 1]['f1_bw'].dropna()
        sex_2_data = df[df['f1_sex'] == 2]['f1_bw'].dropna()
        plt.hist([sex_1_data, sex_2_data], bins=20, alpha=0.7, label=['Sex 1', 'Sex 2'], 
                color=['blue', 'pink'], edgecolor='black')
        plt.xlabel('Birthweight (grams)')
        plt.ylabel('Frequency')
        plt.title('Birthweight Distribution by Sex')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Subplot 3: LBW flag distribution
        plt.subplot(2, 3, 3)
        lbw_counts = df['LBW_flag'].value_counts()
        plt.pie(lbw_counts.values, labels=['Normal BW', 'LBW'], autopct='%1.1f%%', startangle=90)
        plt.title('Low Birthweight Distribution')
        
        # Subplot 4: Box plot by sex
        plt.subplot(2, 3, 4)
        df.boxplot(column='f1_bw', by='f1_sex', ax=plt.gca())
        plt.title('Birthweight by Sex')
        plt.suptitle('')  # Remove automatic title
        
        # Subplot 5: LBW rate by sex
        plt.subplot(2, 3, 5)
        lbw_by_sex = df.groupby('f1_sex')['LBW_flag'].mean() * 100
        bars = plt.bar(['Sex 1', 'Sex 2'], lbw_by_sex.values, color=['blue', 'pink'], edgecolor='black')
        plt.ylabel('LBW Rate (%)')
        plt.title('LBW Rate by Sex')
        for bar, rate in zip(bars, lbw_by_sex.values):
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                    f'{rate:.1f}%', ha='center', va='bottom', fontweight='bold')
        plt.grid(True, alpha=0.3)
        
        # Subplot 6: Maternal age vs birthweight
        plt.subplot(2, 3, 6)
        if 'f0_m_age' in df.columns:
            plt.scatter(df['f0_m_age'], df['f1_bw'], alpha=0.6, s=20)
            plt.xlabel('Maternal Age (years)')
            plt.ylabel('Birthweight (grams)')
            plt.title('Maternal Age vs Birthweight')
            plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(plots_dir / 'comprehensive_birthweight_analysis.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✅ Birthweight analysis plots saved")
    
    # 2. Domain distribution plot
    print("📊 Creating domain distribution plot...")
    
    domain_counts = grouping_df['Domain'].value_counts()
    
    plt.figure(figsize=(12, 8))
    bars = plt.bar(range(len(domain_counts)), domain_counts.values, color='lightcoral', edgecolor='black')
    plt.xlabel('Domain')
    plt.ylabel('Number of Variables')
    plt.title('Variable Distribution by Domain')
    plt.xticks(range(len(domain_counts)), domain_counts.index, rotation=45, ha='right')
    
    # Add value labels on bars
    for bar, count in zip(bars, domain_counts.values):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                str(count), ha='center', va='bottom', fontweight='bold')
    
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(plots_dir / 'domain_distribution_eda.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("✅ Domain distribution plot saved")
    
    # 3. Key variable correlations
    print("📊 Creating correlation heatmap...")
    
    # Get key variables for correlation
    key_vars = []
    for domain in ['Maternal_socio_demographic', 'Maternal_clinical_biomarkers', 'Maternal_anthropometry']:
        domain_vars = grouping_df[grouping_df['Domain'] == domain]['Variable'].tolist()
        domain_vars = [var for var in domain_vars if var in df.columns and df[var].dtype in ['int64', 'float64'] and df[var].nunique() > 1]
        key_vars.extend(domain_vars[:5])  # Take first 5 from each domain
    
    # Add birthweight
    if 'f1_bw' in df.columns:
        key_vars.append('f1_bw')
    
    # Limit to 20 variables for readability
    key_vars = key_vars[:20]
    
    if len(key_vars) > 1:
        plt.figure(figsize=(15, 12))
        correlation_matrix = df[key_vars].corr()
        mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
        sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
                   square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
        plt.title('Correlation Heatmap - Key Variables', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.savefig(plots_dir / 'key_variables_correlation.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✅ Correlation heatmap saved")
    
    print(f"✅ All EDA plots saved to {plots_dir}")


In [ ]:
def create_eda_summary_report(df, grouping_df, correlations, risk_factors):
    """Create EDA summary report"""
    print("\n📊 STEP 5: CREATING EDA SUMMARY REPORT")
    print("-" * 50)
    
    # Create reports directory
    reports_dir = Path('Reports')
    reports_dir.mkdir(exist_ok=True)
    
    # Create summary statistics
    summary_stats = []
    
    # Dataset overview
    summary_stats.append({
        'Category': 'Dataset Overview',
        'Metric': 'Total Variables',
        'Value': df.shape[1],
        'Description': 'Total number of variables in the dataset'
    })
    
    summary_stats.append({
        'Category': 'Dataset Overview',
        'Metric': 'Total Observations',
        'Value': df.shape[0],
        'Description': 'Total number of observations'
    })
    
    summary_stats.append({
        'Category': 'Dataset Overview',
        'Metric': 'Missing Values',
        'Value': df.isnull().sum().sum(),
        'Description': 'Total missing values across all variables'
    })
    
    # Birthweight analysis
    if 'f1_bw' in df.columns:
        summary_stats.append({
            'Category': 'Birthweight Analysis',
            'Metric': 'Mean Birthweight (g)',
            'Value': round(df['f1_bw'].mean(), 2),
            'Description': 'Average birthweight in grams'
        })
        
        summary_stats.append({
            'Category': 'Birthweight Analysis',
            'Metric': 'LBW Rate (%)',
            'Value': round((df['LBW_flag'].sum() / len(df)) * 100, 2),
            'Description': 'Percentage of low birthweight cases'
        })
    
    # Domain analysis
    domain_counts = grouping_df['Domain'].value_counts()
    for domain, count in domain_counts.items():
        summary_stats.append({
            'Category': 'Domain Analysis',
            'Metric': f'{domain} Variables',
            'Value': count,
            'Description': f'Number of variables in {domain} domain'
        })
    
    # Correlation analysis
    significant_correlations = [var for var, stats in correlations.items() if abs(stats['correlation']) > 0.3]
    summary_stats.append({
        'Category': 'Correlation Analysis',
        'Metric': 'Strong Correlations (>0.3)',
        'Value': len(significant_correlations),
        'Description': 'Variables with strong correlation to birthweight'
    })
    
    # Risk factor analysis
    significant_risk_factors = [var for var, stats in risk_factors.items() if stats['p_value'] < 0.05]
    summary_stats.append({
        'Category': 'Risk Factor Analysis',
        'Metric': 'Significant Risk Factors',
        'Value': len(significant_risk_factors),
        'Description': 'Variables significantly associated with LBW'
    })
    
    # Create DataFrame and save
    summary_df = pd.DataFrame(summary_stats)
    report_file = reports_dir / 'eda_summary_report.csv'
    summary_df.to_csv(report_file, index=False)
    
    print(f"✅ EDA summary report saved: {report_file}")
    
    # Display summary
    print("\n📈 EDA Summary Report:")
    print(summary_df.to_string(index=False))
    
    return report_file


In [ ]:
def main():
    """Main execution function"""
    # Load processed data
    df, grouping_df = load_processed_data()
    if df is None:
        return
    
    # Analyze each domain
    domains = ['Maternal_socio_demographic', 'Maternal_clinical_biomarkers', 
              'Maternal_anthropometry', 'Household_environment', 'Child_outcomes']
    
    domain_analyses = {}
    for domain in domains:
        domain_analyses[domain] = analyze_domain_variables(df, grouping_df, domain)
    
    # Correlation and risk factor analysis
    correlations = correlation_analysis_with_birthweight(df)
    risk_factors = risk_factor_analysis(df)
    
    # Create visualizations
    create_eda_plots(df, grouping_df)
    
    # Create summary report
    report_file = create_eda_summary_report(df, grouping_df, correlations, risk_factors)
    
    # Final summary
    print("\n" + "="*80)
    print("SIMPLIFIED EDA COMPLETE")
    print("="*80)
    print("✅ Analysis completed:")
    print("  - Domain-wise variable analysis")
    print("  - Correlation analysis with birthweight")
    print("  - Risk factor analysis for LBW")
    print("  - Comprehensive visualizations created")
    print("  - EDA summary report generated")
    print("="*80)


In [ ]:
if __name__ == "__main__":
    main()
